<a href="https://colab.research.google.com/github/bhav380-2/Deep-Learning/blob/main/Fine_Tuning_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from tqdm import tqdm
import re
import json
import matplotlib.pyplot as plt
import json
import math

import math
import re
from   random import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os

# Fine tuning BERT

## Load Dataset

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("imdb")
print(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


## Toknize Dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
sample_text = "I love this movie , I recommend watching it!"
tokens = tokenizer(sample_text,padding="max_length",truncation=True,max_length=128)
print(tokens)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'input_ids': [101, 1045, 2293, 2023, 3185, 1010, 1045, 16755, 3666, 2009, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
def tokenize_function(examples):
  return tokenizer(examples['text'],padding="max_length",truncation=True,max_length=128)

tokenized_datasets = dataset.map(tokenize_function,batched=True)
print(tokenized_datasets["train"][0])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

## Load BERT

In [ ]:
from transformers import AutoModelForSequenceClassification
model_name = 'bert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
print(model.config)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: bert-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
classifier.weight                          | MISSING    | 
classifier.bias                            | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


BertConfig {
  "add_cross_attention": false,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "classifier_dropout": null,
  "dtype": "float32",
  "eos_token_id": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tie_word_embeddings": true,
  "transformers_version": "5.0.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



## Freezing all layers except classifier

In [ ]:
for param in model.bert.parameters():
  param.requires_grad = False

for param in model.classifier.parameters():
  param.requires_grad = True

print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

Trainable parameters: 1538


## Extending BERT's Architecture

In [ ]:
class CustomBERTModel(nn.Module):
  def __init__(self,pretrained_model_name,num_labels):
    super(CustomBERTModel,self).__init__()
    self.bert = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name,num_labels=num_labels)
    self.dropout = nn.Dropout(0.3)
    self.fc = nn.Linear(self.bert.config.hidden_size,num_labels)

  def forward(self,input_ids,attention_mask):
    output = self.bert(input_ids=input_ids,attention_mask=attention_mask)
    pooled_output = self.dropout(output[1])
    logits = self.fc(pooled_output)
    return logits

custom_model = CustomBERTModel('bert-base-uncased',num_labels=3)

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: bert-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
classifier.weight                          | MISSING    | 
classifier.bias                            | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


## Define Training Arguments

In [ ]:


from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir = "./results",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 5e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 3,
    weight_decay = 0.01,
    save_total_limit=2,
    load_best_model_at_end =True,
    logging_dir="./logs",
    logging_steps=100,
    fp16=True
)

print(training_args)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


TrainingArguments(
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=True,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
enable_jit_checkpoint=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
eval_use_gather_object=Fals

## Custom metric for Eval

In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
from evaluate import load
metric = load("f1")

def compute_metrics(eval_pred):
  logits,labels = eval_pred
  predictions = logits.argmax(axis=-1)
  return metric.compute(predictions=predictions,references=labels)



## Data Collation

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Trainer

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test'],
    data_collator= data_collator,
    compute_metrics = compute_metrics
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


In [ ]:
# Trainer automatically handles gradient clipping, saving checkpoints
# and evaluating model -> helping us to focus on finetuning rather
# than debuggging boilerplate code

## Techniques

In [ ]:
# accumulating gradients
training_args.gradient_accumulation_steps = 4 # when GPU can't handle large batch sizes

# Early stopping
from transformers import EarlyStoppingCallback
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))


# Dynamic learning Rate schedulers
from transformers import get_scheduler
scheduler = get_scheduler(
    "linear",
    optimizer = trainer.optimizer,
    num_warmup_steps=0,
    num_training_steps = len(trainer.train_dataloader()) * training_args.num_train_epochs
)


In [ ]:
# Distributed Training if have access to multiple GPU's

# python -m torch.distributed.launch --nproc_per_node=4 train.py

## Practical Solutions
1. Overfitting -> increase regularization (weight_decay) or apply dropout layers.
Reduce number of epochs

2. Class Imbalance -> Use weighted loss functions or oversampling.
In hugging face transformer implement Custom loss function

3. Slow Traning -> reduce max_length during tokenization.
Enable fp16 for faster computations

## Evaluate Model

In [ ]:
# to truly understand model's strength and weaknesses
# deep diver into metrics like F1-score, precision-recall
# curves and confusion matrix

In [ ]:
# Quick evaluation
results = trainer.evaluate()
print(results)

In [ ]:
# Precision-Recall curvie and Confusion Matrix
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
predictions = trainer.predict(tokenized_datasets['test'])
predicted_labels = predictions.predictions.argmax(axis=-1)

#Classification Report
print(classification_report(tokenized_datasets['test']['label'],predicted_labels))

# Confusion Matrix
cm = confusion_matrix(tokenized_datasets['test']['label'], predicted_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Class 0',"Class 1"])
disp.plot()

## Error Analysis

In [ ]:
## Identify patterns in misclassifed samples

# inspect misclassified samples
for idx, (pred, label) in enumerate(zip(predicted_labels,tokenized_datasets['test']['label'])):
  if idx==100:
    break
  if pred!=label:
    print(f"Index: {idx}, predicted:{pred}, Actual: {label}")
    print(tokenized_datasets["test"][idx]["text"])

In [ ]:
import shap
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

# Define a prediction function that SHAP can use
# This function takes raw text inputs and returns model outputs (logits)
def predict_function(texts):
    # Ensure texts is a list of strings, handling potential numpy array inputs from SHAP
    if isinstance(texts, str):
        texts = [texts]
    elif isinstance(texts, np.ndarray):
        texts = texts.tolist() # Convert numpy array of strings to a list of strings

    # Tokenize the input texts
    # 'padding' and 'truncation' are important for consistent input size
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors='pt')

    # Move inputs to the correct device where the model is
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Get model outputs (logits)
    with torch.no_grad():
        # Pass the dictionary of inputs directly to the model
        outputs = model(**inputs)

    # Return logits as a NumPy array for SHAP
    return outputs.logits.cpu().numpy()

# Initialize the SHAP explainer with the prediction function
# When using a prediction function, the tokenizer is often passed as a separate argument
# to help SHAP understand the text structure, even though the function itself tokenizes.
explainer = shap.Explainer(predict_function, tokenizer)

# Generate SHAP values for a subset of the test dataset
# Pass raw text to the explainer, which will then use predict_function
shap_values = explainer(tokenized_datasets['test']['text'][:10])

# Plot the explanation for the first example
shap.plots.text(shap_values[0])

## Advance Fine Tuning

### Hperparameter Tuning


In [ ]:
import optuna

def objective(trial):
  learning_rate = trial.suggest_float("learning_rate",1e-5,5e-4,log=True)
  batch_size = trial.suggest_categorical("batch_size",[16,32,64])
  num_train_epochs = trial.suggest_int("num_train_epochs",2,5)

  training_args = TrainingArguments(
      learning_rate = learning_rate,
      per_device_train_batch_size = batch_size,
      num_train_epochs = num_train_epochs,
      outpu_dir = "./results"
  )

  trainer = Trainer(
      model = model,
      args = training_args,
      train_dataset = tokenized_datasets['train'],
      eval_dataset = tokenized_datasets['test']
  )
  result = trainer.evaluate()
  return result['eval_f1']

study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trails=10)

print(study.best_params)



### Handling Large Datasets

In [ ]:
# Data Loaders
from torch.utils.data import DataLoader
train_loader = DataLoader(tokenized_datasets['train'],batch_size=16, shuffle=True)
for batch in train_loader:
  print(batch)



### Domain Adaptation

In [ ]:
# Pretraining BERT on Domain Corpus before fine-turning it on classification task
from transformers import DataCollatorForLanguageModelling
collator = DataCollatorForLanguageModelling(tokenizer=tokenizer, mlm = True, mlm_probability=0.15)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = domain_dataset,
    data_collator = collator
)

trainer.train()

### Extending BERT's Architecture

In [ ]:
import torch.nn as nn

class EnhancedBERT(nn.Module):
  def __init__(self, pretrained_model_name, num_labels):
    super(EnhancedBERT, self).__init__()
    self.bert = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name)
    self.attention = nn.Linear(self.bert.config.hidden_size, 1)
    self.dropout = nn.Droput(0.3)
    self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

  def forward(self, input_ids, attention_mask):
    outputs = self.bert(input_ids, attention_mask).last_hidden_state
    weights = torch.softmax(self.attention(outputs),dim=1)
    pooled_output = (weights *outputs).sum(dim=1)
    logits = self.classifier(self.dropout(pooled_output))
    return logits

## Deployment

### Use ONNX or TorchScript

In [ ]:
# Exporting the model to format like ONNX or TorchScript can significantly
# speed up performance

import torch
from transformers import AutoModelForSequenceClassification

# Load the fine-tuned model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

# Define dummy input for tracing
dummy_input = torch.randint(0, 100, (1, 128))

# Export to ONNX
torch.onnx.export(
    model,
    dummy_input,
    "bert_classification.onnx",
    input_names=["input_ids"],
    output_names=["output"],
    dynamic_axes={"input_ids": {0: "batch_size"}, "output": {0: "batch_size"}},
    opset_version=11
)

print("Model exported to ONNX format")

In [ ]:
# Script the model
scripted_model = torch.jit.script(model)

# Save the scripted model
scripted_model.save("bert_classification.pt")
print("Model exported to TorchScript")

### Hugging Face Pipeline for Quick Deployment

In [ ]:
from transformers import pipeline
classifier = pipeline('text-classification', model='bert-base-uncased')
text = 'I loved this movie! It was fantastic.'
result = classifier(text)
print(result)

### Deployment with FastApi or Flask

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import pipeline

app = FastAPI()

# Load the pipeline
classifier = pipeline("text-classification", model="bert-base-uncased")

# Define the input schema
class TextRequest(BaseModel):
    text: str

@app.post("/predict")
def predict(request: TextRequest):
    result = classifier(request.text)
    return {"label": result[0]["label"], "score": result[0]["score"]}

# Run the app
# uvicorn filename:app --reload

## Common Pitfalls

### Overfitting on small datasets

1. Increase Regularization -> Add or increase weight decay in training args
2. Apply Dropout -> use custom architectures with dropout layers
3. Early Stopping -> Enable early stopping to halt training when validation performance stops improving


4. Gradient Explosion / Vanishing -> Enable Gradient Clipping & Monitor Learning Rate



### CUDA/GPU Errors

1. check GPU memory Usage -> torch.cuda.memory_summary()

2. Reduce Batch Size -> simplest way to fit your model into memory

3. Clear cache:- torch.cuda.empty_cache()




### Optimizing Training Time

1. Use Mixed Precision
2. Pre-tokenize Data
3. Cache Datasets

## Advanced Resources

1. Hugging Face forums
2. Research Papers
3. Hugging face official blogs and github examples
4. Document your own workflows